In [1]:
import pandas as pd

In [2]:
PlayTimeGenero=pd.read_csv("PlayTimeGenre.csv")
reviews=pd.read_csv("Merge_reviews_items.csv")
sentiment=pd.read_csv("Sentiment.csv")
UserNot=pd.read_csv("Recc_O_No.csv")
UserForGenero=pd.read_csv("UserForGenero.csv")

In [167]:
reviews = reviews.dropna()

FUNCION def PlayTimeGenre(genero)

In [7]:
def PlayTimeGenre(genero):
    filtered_df = PlayTimeGenero[PlayTimeGenero['genres'].str.contains(genero, case=False, na=False)]

    if filtered_df.empty:
        return f"No se encontraron registros para el género '{genero}'"

    max_playtime_year = filtered_df.groupby('release_date')['playtime_forever'].sum().idxmax()

    return {"Año de lanzamiento con más horas jugadas para el género " + genero: max_playtime_year}


genero_buscado = "Action"  # Cambiar esto al género que desees buscar
resultado = PlayTimeGenre(genero_buscado)
print(resultado)

{'Año de lanzamiento con más horas jugadas para el género Action': 2012}


Funcion def UserForGenre(merged_df, genero)

In [18]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado UserForGenero

def UserForGenre(genero):
    # Filtrar el DataFrame por el género deseado
    df_genero = UserForGenero[UserForGenero['genres'].str.contains(genero, case=False)]
    
    if df_genero.empty:
        return f"No se encontraron registros para el género '{genero}'"
    
    # Agrupar por usuario y año, sumando las horas jugadas
    df_agrupado = df_genero.groupby(['user_id', 'posted'])['playtime_forever'].sum().reset_index()
    
    # Encontrar al usuario con más horas jugadas para el género
    max_user = df_agrupado.loc[df_agrupado.groupby('user_id')['playtime_forever'].idxmax()]
    
    # Crear la lista de acumulación de horas jugadas por año
    horas_por_anio = [{"Año": año, "Horas": horas} for año, horas in zip(max_user['posted'], max_user['playtime_forever'])]
    
    return {"Usuario con más horas jugadas para Género " + genero: max_user['user_id'].values[0], "Horas jugadas": horas_por_anio}

{'Usuario con más horas jugadas para Género Indie': '--ace--', 'Horas jugadas': [{'Año': 2014, 'Horas': 1494.0}, {'Año': 2015, 'Horas': 772.0}, {'Año': 2015, 'Horas': 454.0}, {'Año': 2014, 'Horas': 165.0}, {'Año': 2014, 'Horas': 412.0}, {'Año': 2014, 'Horas': 1005.0}, {'Año': 2014, 'Horas': 309.0}, {'Año': 2015, 'Horas': 407.0}, {'Año': 2013, 'Horas': 412.0}, {'Año': 2014, 'Horas': 327.0}, {'Año': 2013, 'Horas': 67.0}, {'Año': 2014, 'Horas': 1005.0}, {'Año': 2015, 'Horas': 54.0}, {'Año': 2014, 'Horas': 790.0}, {'Año': 2014, 'Horas': 219.0}, {'Año': 2014, 'Horas': 144.0}, {'Año': 2014, 'Horas': 997.0}, {'Año': 2015, 'Horas': 417.0}, {'Año': 2014, 'Horas': 67.0}, {'Año': 2013, 'Horas': 954.0}, {'Año': 2015, 'Horas': 412.0}, {'Año': 2014, 'Horas': 718.0}, {'Año': 2014, 'Horas': 44.0}, {'Año': 2014, 'Horas': 292.0}, {'Año': 2014, 'Horas': 412.0}, {'Año': 2014, 'Horas': 4254.0}, {'Año': 2014, 'Horas': 54.0}, {'Año': 2014, 'Horas': 54.0}, {'Año': 2014, 'Horas': 38.0}, {'Año': 2015, 'Horas': 

In [6]:
resultado = UserForGenre("Action", UserForGenero,)
print(resultado)

No se encontraron registros para el género 'Action'


Funcion def UsersRecommend( año : int)

In [126]:
def UsersRecommend(año: int):
    # Filtrar el DataFrame por el año y las condiciones de recomendación y sentimiento
    filtro = (reviews['posted'] == año) & (reviews['recommend'] == True) & (reviews['sentiment_analysis'] >= 0)
    juegos_recomendados = reviews[filtro]

    # Verificar si hay datos para el año especificado
    if juegos_recomendados.empty:
        return "Año no encontrado"

    # Contar la cantidad de reseñas por juego
    juegos_count = juegos_recomendados['app_name'].value_counts()

    # Obtener los tres juegos más recomendados
    top_3_juegos = juegos_count.head(3)

    # Crear una lista de resultados en el formato deseado
    resultado = []
    for puesto, juego in enumerate(top_3_juegos.index, 1):
        resultado.append({f"Puesto {puesto}": juego})

    return resultado

In [129]:
resultados = UsersRecommend(2010)
print(resultados)


[{'Puesto 1': 'Team Fortress 2'}, {'Puesto 2': 'Killing Floor'}, {'Puesto 3': 'Alien Swarm'}]


Funcion def UsersNotRecommend( año : int )

In [205]:
def UsersNotRecommend(year):
    # Obtiene los años únicos disponibles en la columna 'posted'
    años_disponibles = UserNot['posted'].unique()

    # Verifica si el año proporcionado está en la lista de años disponibles
    if year not in años_disponibles:
        return "Año no encontrado"

    # Filtrar el DataFrame para el año dado y donde 'recommend' es False
    filtered_df = UserNot[(UserNot['posted'] == year) & (UserNot['recommend'] == False)]

    # Contar la cantidad de juegos menos recomendados para cada juego
    game_counts = filtered_df['app_name'].value_counts()

    # Tomar los 3 juegos menos recomendados
    top_3_least_recommended = game_counts.tail(3)

    # Crear una lista de diccionarios en el formato deseado
    result = [{"Puesto {}".format(i + 1): game} for i, (game, count) in enumerate(top_3_least_recommended.items())]

    return result

FUNCION SENTIMENT_ANALISIS

In [38]:
def sentiment_analysis(dataframe, year):
    if year not in dataframe['posted'].astype(int).unique():
        return "Año no encontrado"
    # Convertir la columna 'posted' a tipo int64 para la comparación
    dataframe['posted'] = dataframe['posted'].astype(int)
    # Filtrar el DataFrame para obtener las filas correspondientes al año dado
    year_df = dataframe[dataframe['posted'] == year]
    # Calcular la cantidad de registros para cada categoría de sentimiento
    sentiment_counts = year_df['sentiment_analysis'].value_counts().to_dict()
    # Crear el diccionario de respuesta
    respuesta = {
        'Negative': sentiment_counts.get(0, 0),  # 0 representa 'Negative'
        'Neutral': sentiment_counts.get(1, 0),   # 1 representa 'Neutral'
        'Positive': sentiment_counts.get(2, 0)   # 2 representa 'Positive'
    }
    return respuesta

In [36]:
tipos_de_datos = sentiment.dtypes

# Mostrar los tipos de datos
print(tipos_de_datos)

sentiment_analysis    int64
posted                int64
dtype: object


In [40]:
resultado = sentiment_analysis(sentiment, 2019)  # Reemplaza 2015 con el año que desees
print(resultado)

Año no encontrado
